In [7]:
#dependencies 
#had to 
import pandas as pd 
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import config 
from coincallandwritepythonfile import refined_coin_list

In [8]:
#still have issue with a blank line showing 
refined_coin_list

,id,name,symbol,rank,File_Name,injest_datetime
1,1,Bitcoin,BTC,1,Coin_Ranking_Dim,04/22/2022 10:04:00
2,1027,Ethereum,ETH,2,Coin_Ranking_Dim,04/22/2022 10:04:00
3,825,Tether,USDT,3,Coin_Ranking_Dim,04/22/2022 10:04:00
4,1839,BNB,BNB,4,Coin_Ranking_Dim,04/22/2022 10:04:00
5,3408,USD Coin,USDC,5,Coin_Ranking_Dim,04/22/2022 10:04:00
...,...,...,...,...,...,...
97,1896,0x,ZRX,97,Coin_Ranking_Dim,04/22/2022 10:04:00
98,7455,Audius,AUDIO,98,Coin_Ranking_Dim,04/22/2022 10:04:00
99,2099,ICON,ICX,99,Coin_Ranking_Dim,04/22/2022 10:04:00
100,3801,BORA,BORA,100,Coin_Ranking_Dim,04/22/2022 10:04:00


In [9]:
#get ids for each coin and convert to list
coin_ids = refined_coin_list.id.to_list()
type(coin_ids)
#getting 5 first ids to try in practice run
five_ids_to_try = coin_ids[:5]

In [16]:
def get_coin_info_reorder(list_of_coins_, api_key):
    df = pd.DataFrame()
    for id in list_of_coins_:
        url = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/quotes/latest'
        parameters = {
            'id': id
             }
        headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': api_key
        }

        session = Session()
        session.headers.update(headers)
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        #navigate json
        #read to DF to see what we have
        cmc_df = pd.DataFrame.from_dict(data['data'], orient='index')
        df = df.append(cmc_df)
        
        #expanded_df = pd.DataFrame.from_dict(cmc_df['quote'][0], orient= 'index')
        #showing USD as an index column, going to drop
        
                       
        #trial = cmc_df.drop(['quote','tags'], axis= 1)
        #had to reset index or set it manually to match the expanded DF 
    
        
        
        #had to add key to get them to append properly
        #coin_price_df = pd.concat([trial,expanded_df], axis=1, keys=['trial', 'expanded_df']  )
        #coin_price_df.reset_index(drop = True, inplace= True)

        #this saves it as a coreframe

        #going to try to append this to another dataframe
        #df = pd.DataFrame()
        #coin_price_df.set_index(pd.Index([count]),inplace= True)

        #df = df.append(coin_price_df)
        #df.set_index(pd.Index([count]),inplace= True)
    #had to drop level inorder to get ride of multilevel index   
    # #https://sparkbyexamples.com/pandas/pandas-multiindex-dataframe-examples/?msclkid=fba45bb6c10e11eca1d376f5b274bab7 
    #df = df.droplevel(0, axis =1)
    return df

        


In [18]:
df = get_coin_info_reorder(five_ids_to_try, config.coin_api_key)

In [120]:
expanded_df = df[['platform','quote']].apply(pd.Series)


In [121]:
quote_df =pd.DataFrame.from_dict(expanded_df['quote'][0], orient= 'index')

In [125]:
expanded_df['quote'].explode()

1       USD
1027    USD
825     USD
1839    USD
3408    USD
Name: quote, dtype: object

In [122]:
quote_df

,price,volume_24h,volume_change_24h,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,last_updated
USD,39523.796835,3.225925e+10,7.1626,0.354209,-5.358801,-2.440471,-6.942925,1.871858,13.14049,7.517023e+11,40.7455,8.299997e+11,2022-04-22T17:18:00.000Z


In [114]:
df.drop(['quote', 'tags','platform'], axis= 1)


,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,is_active,platform,cmc_rank,is_fiat,self_reported_circulating_supply,self_reported_market_cap,last_updated,quote
1,1,Bitcoin,BTC,bitcoin,9355,2013-04-28T00:00:00.000Z,"[{'slug': 'mineable', 'name': 'Mineable', 'cat...",21000000,1.901898e+07,1.901898e+07,1,None,1,0,None,None,2022-04-22T17:18:00.000Z,"{'USD': {'price': 39523.796835383204, 'volume_..."
1027,1027,Ethereum,ETH,ethereum,5659,2015-08-07T00:00:00.000Z,"[{'slug': 'mineable', 'name': 'Mineable', 'cat...",None,1.204841e+08,1.204841e+08,1,None,2,0,None,None,2022-04-22T17:17:00.000Z,"{'USD': {'price': 2958.5534929425485, 'volume_..."
825,825,Tether,USDT,tether,32216,2015-02-25T00:00:00.000Z,"[{'slug': 'payments', 'name': 'Payments', 'cat...",None,8.306920e+10,8.561395e+10,1,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",3,0,None,None,2022-04-22T17:17:00.000Z,"{'USD': {'price': 1.0002524796835452, 'volume_..."
1839,1839,BNB,BNB,bnb,812,2017-07-25T00:00:00.000Z,"[{'slug': 'marketplace', 'name': 'Marketplace'...",165116760,1.632770e+08,1.632770e+08,1,None,4,0,None,None,2022-04-22T17:17:00.000Z,"{'USD': {'price': 407.8732987541661, 'volume_2..."
3408,3408,USD Coin,USDC,usd-coin,3790,2018-10-08T00:00:00.000Z,"[{'slug': 'medium-of-exchange', 'name': 'Mediu...",None,4.989235e+10,4.989235e+10,1,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",5,0,None,None,2022-04-22T17:17:00.000Z,"{'USD': {'price': 1.0001236643976918, 'volume_..."


In [52]:
#while loop to expand out all of the columns 
i = 0
concat_df = pd.DataFrame()
while i <= 10:
    i+=1
    tags = df['tags'].apply(pd.Series)
    exploded_df = tags[i].apply(pd.Series)
concat_df.append(exploded_df)

,0,category,name,slug
1,NaN,PROPERTY,CMS Holdings Portfolio,cms-holdings-portfolio
1027,NaN,PROPERTY,DCG Portfolio,dcg-portfolio
825,NaN,NaN,NaN,NaN
1839,NaN,NaN,NaN,NaN
3408,NaN,NaN,NaN,NaN


In [71]:
i = 0
number_of_columns = []
while i < len(tags.columns):
    number_of_columns.append(i)
    i+= 1

In [84]:
#while loop to expand out all of the columns 
i = 0
concat_df = pd.DataFrame()
number_of_columns = []
tags = df['tags'].apply(pd.Series)
while i < len(tags.columns):
    number_of_columns.append(i)
    i+=1


In [99]:
tags

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
1,"{'slug': 'mineable', 'name': 'Mineable', 'cate...","{'slug': 'pow', 'name': 'PoW', 'category': 'CO...","{'slug': 'sha-256', 'name': 'SHA-256', 'catego...","{'slug': 'store-of-value', 'name': 'Store Of V...","{'slug': 'state-channel', 'name': 'State Chann...","{'slug': 'coinbase-ventures-portfolio', 'name'...","{'slug': 'three-arrows-capital-portfolio', 'na...","{'slug': 'polychain-capital-portfolio', 'name'...","{'slug': 'binance-labs-portfolio', 'name': 'Bi...","{'slug': 'blockchain-capital-portfolio', 'name...",...,"{'slug': 'a16z-portfolio', 'name': 'a16z Portf...","{'slug': '1confirmation-portfolio', 'name': '1...","{'slug': 'winklevoss-capital-portfolio', 'name...","{'slug': 'usv-portfolio', 'name': 'USV Portfol...","{'slug': 'placeholder-ventures-portfolio', 'na...","{'slug': 'pantera-capital-portfolio', 'name': ...","{'slug': 'multicoin-capital-portfolio', 'name'...","{'slug': 'paradigm-portfolio', 'name': 'Paradi...",NaN,NaN
1027,"{'slug': 'mineable', 'name': 'Mineable', 'cate...","{'slug': 'pow', 'name': 'PoW', 'category': 'CO...","{'slug': 'smart-contracts', 'name': 'Smart Con...","{'slug': 'ethereum-ecosystem', 'name': 'Ethere...","{'slug': 'coinbase-ventures-portfolio', 'name'...","{'slug': 'three-arrows-capital-portfolio', 'na...","{'slug': 'polychain-capital-portfolio', 'name'...","{'slug': 'binance-labs-portfolio', 'name': 'Bi...","{'slug': 'blockchain-capital-portfolio', 'name...","{'slug': 'boostvc-portfolio', 'name': 'BoostVC...",...,"{'slug': 'a16z-portfolio', 'name': 'a16z Portf...","{'slug': '1confirmation-portfolio', 'name': '1...","{'slug': 'winklevoss-capital-portfolio', 'name...","{'slug': 'usv-portfolio', 'name': 'USV Portfol...","{'slug': 'placeholder-ventures-portfolio', 'na...","{'slug': 'pantera-capital-portfolio', 'name': ...","{'slug': 'multicoin-capital-portfolio', 'name'...","{'slug': 'paradigm-portfolio', 'name': 'Paradi...","{'slug': 'injective-ecosystem', 'name': 'Injec...","{'slug': 'bnb-chain', 'name': 'BNB Chain', 'ca..."
825,"{'slug': 'payments', 'name': 'Payments', 'cate...","{'slug': 'stablecoin', 'name': 'Stablecoin', '...","{'slug': 'asset-backed-stablecoin', 'name': 'A...","{'slug': 'avalanche-ecosystem', 'name': 'Avala...","{'slug': 'solana-ecosystem', 'name': 'Solana E...","{'slug': 'arbitrum-ecosytem', 'name': 'Arbitru...","{'slug': 'moonriver-ecosystem', 'name': 'Moonr...","{'slug': 'injective-ecosystem', 'name': 'Injec...","{'slug': 'bnb-chain', 'name': 'BNB Chain', 'ca...","{'slug': 'usd-stablecoin', 'name': 'USD Stable...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1839,"{'slug': 'marketplace', 'name': 'Marketplace',...","{'slug': 'centralized-exchange', 'name': 'Cent...","{'slug': 'payments', 'name': 'Payments', 'cate...","{'slug': 'smart-contracts', 'name': 'Smart Con...","{'slug': 'alameda-research-portfolio', 'name':...","{'slug': 'multicoin-capital-portfolio', 'name'...","{'slug': 'moonriver-ecosystem', 'name': 'Moonr...","{'slug': 'bnb-chain', 'name': 'BNB Chain', 'ca...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3408,"{'slug': 'medium-of-exchange', 'name': 'Medium...","{'slug': 'stablecoin', 'name': 'Stablecoin', '...","{'slug': 'asset-backed-stablecoin', 'name': 'A...","{'slug': 'fantom-ecosystem', 'name': 'Fantom E...","{'slug': 'arbitrum-ecosytem', 'name': 'Arbitru...","{'slug': 'moonriver-ecosystem', 'name': 'Moonr...","{'slug': 'bnb-chain', 'name': 'BNB Chain', 'ca...","{'slug': 'usd-stablecoin', 'name': 'USD Stable...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
for column in number_of_columns:
    exploded_df = tags[column].apply(pd.Series)
    cols = exploded_df.columns
    cols_new = ["{}_{}_{}".format(x,i,"Expanded") for x in cols]

In [109]:
cols_new

['0_30_Expanded',
 'category_30_Expanded',
 'name_30_Expanded',
 'slug_30_Expanded']

In [112]:
#returns a lot of columns, will need to rename as well
concat_df = pd.DataFrame()
for column in number_of_columns:
    exploded_df = tags[column].apply(pd.Series)
    cols = exploded_df.columns
    cols_new = ["{}_{}_{}".format(x,column,"Expanded") for x in cols]
    mapping = {key1: key2 for key1, key2 in zip(cols, cols_new)}
    exploded_df = exploded_df.rename(columns=mapping)
    concat_df = pd.concat([exploded_df, concat_df], axis=1)    

In [113]:
concat_df

,0_29_Expanded,category_29_Expanded,name_29_Expanded,slug_29_Expanded,0_28_Expanded,category_28_Expanded,name_28_Expanded,slug_28_Expanded,0_27_Expanded,category_27_Expanded,...,category_3_Expanded,slug_2_Expanded,name_2_Expanded,category_2_Expanded,slug_1_Expanded,name_1_Expanded,category_1_Expanded,slug_0_Expanded,name_0_Expanded,category_0_Expanded
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PROPERTY,...,PROPERTY,sha-256,SHA-256,CONSENSUS_ALGORITHM,pow,PoW,CONSENSUS_ALGORITHM,mineable,Mineable,OTHER
1027,NaN,PROPERTY,BNB Chain,bnb-chain,NaN,PROPERTY,Injective Ecosystem,injective-ecosystem,NaN,PROPERTY,...,PLATFORM,smart-contracts,Smart Contracts,PROPERTY,pow,PoW,CONSENSUS_ALGORITHM,mineable,Mineable,OTHER
825,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,PROPERTY,asset-backed-stablecoin,Asset-Backed Stablecoin,PROPERTY,stablecoin,Stablecoin,PROPERTY,payments,Payments,PROPERTY
1839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,PROPERTY,payments,Payments,PROPERTY,centralized-exchange,Centralized Exchange,PROPERTY,marketplace,Marketplace,PROPERTY
3408,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,PROPERTY,asset-backed-stablecoin,Asset-Backed Stablecoin,PROPERTY,stablecoin,Stablecoin,PROPERTY,medium-of-exchange,Medium of Exchange,PROPERTY


In [33]:
tags = df['tags'].apply(pd.Series)
tags.columns
tags[0].apply(pd.Series)

,slug,name,category
1,mineable,Mineable,OTHER
1027,mineable,Mineable,OTHER
825,payments,Payments,PROPERTY
1839,marketplace,Marketplace,PROPERTY
3408,medium-of-exchange,Medium of Exchange,PROPERTY


In [24]:
tags = df['tags'].apply(pd.Series)
tags[0]
trial_cleaning = pd.concat([df.drop(['tags'], axis= 1),tags[0]], axis= 1 )
trial_cleaning

,id,name,symbol,slug,num_market_pairs,date_added,max_supply,circulating_supply,total_supply,is_active,platform,cmc_rank,is_fiat,self_reported_circulating_supply,self_reported_market_cap,last_updated,quote,0
1,1,Bitcoin,BTC,bitcoin,9355,2013-04-28T00:00:00.000Z,21000000,1.901898e+07,1.901898e+07,1,None,1,0,None,None,2022-04-22T17:18:00.000Z,"{'USD': {'price': 39523.796835383204, 'volume_...","{'slug': 'mineable', 'name': 'Mineable', 'cate..."
1027,1027,Ethereum,ETH,ethereum,5659,2015-08-07T00:00:00.000Z,None,1.204841e+08,1.204841e+08,1,None,2,0,None,None,2022-04-22T17:17:00.000Z,"{'USD': {'price': 2958.5534929425485, 'volume_...","{'slug': 'mineable', 'name': 'Mineable', 'cate..."
825,825,Tether,USDT,tether,32216,2015-02-25T00:00:00.000Z,None,8.306920e+10,8.561395e+10,1,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",3,0,None,None,2022-04-22T17:17:00.000Z,"{'USD': {'price': 1.0002524796835452, 'volume_...","{'slug': 'payments', 'name': 'Payments', 'cate..."
1839,1839,BNB,BNB,bnb,812,2017-07-25T00:00:00.000Z,165116760,1.632770e+08,1.632770e+08,1,None,4,0,None,None,2022-04-22T17:17:00.000Z,"{'USD': {'price': 407.8732987541661, 'volume_2...","{'slug': 'marketplace', 'name': 'Marketplace',..."
3408,3408,USD Coin,USDC,usd-coin,3790,2018-10-08T00:00:00.000Z,None,4.989235e+10,4.989235e+10,1,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",5,0,None,None,2022-04-22T17:17:00.000Z,"{'USD': {'price': 1.0001236643976918, 'volume_...","{'slug': 'medium-of-exchange', 'name': 'Medium..."


In [127]:
        url = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/quotes/latest'
        parameters = {
            'id': five_ids_to_try            
            }
        headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': config.coin_api_key
        }

        session = Session()
        session.headers.update(headers)
        response = session.get(url, params=parameters)
        data = json.loads(response.text)

In [128]:
data

{'status': {'timestamp': '2022-04-22T21:39:20.715Z',
  'error_code': 400,
  'error_message': '"id" must be a string',
  'elapsed': 0,
  'credit_count': 0}}

In [137]:
def get_coin_info(list_of_coins_, api_key):
    """Function makes API call for each coin ID within list of coins.
    Designed to take list from top 100 coins and get their most recent price information. """

    #create an empty dataframe that we will append responses too
    df = pd.DataFrame()
    #a count variable to index the respones when trying to concat 
    count =0
    for id in list_of_coins_:
        url = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/quotes/latest'
        parameters = {
            'id': id
            }
        headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': api_key
        }

        #make API call and return response as a dictionary
        session = Session()
        session.headers.update(headers)
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        
        #parse dict and load in relavant data to a DF
        cmc_df = pd.DataFrame.from_dict(data['data'], orient='index')
         
        #expand nested quote column containing price information
        expanded_df = pd.DataFrame.from_dict(cmc_df['quote'][0], orient= 'index')
        

        
        #set then index for count ie the number of runs 
        expanded_df.set_index(pd.Index([count]),inplace= True)

        #dropping nested columns that we won't explore during this exercise
        trial = cmc_df.drop(['quote','tags', 'platform'], axis= 1)
          
        #setting index as count variable so can join back
        trial.set_index(pd.Index([count]),inplace= True)
        
        #concat was tricky had to add
        #had to add key to get them to append properly
        coin_price_df = pd.concat([trial,expanded_df], axis=1, keys=['trial', 'expanded_df']  )
        #coin_price_df.reset_index(drop = True, inplace= True)
        coin_price_merge = pd.merge(trial, expanded_df)
        #this saves it as a coreframe

        #going to try to append this to another dataframe
        #df = pd.DataFrame()
        #coin_price_df.set_index(pd.Index([count]),inplace= True)
        
        df = df.append(coin_price_merge)

        #df = df.append(coin_price_df)


        count +=1
        #df.set_index(pd.Index([count]),inplace= True)
    #had to drop level inorder to get ride of multilevel index   
    # #https://sparkbyexamples.com/pandas/pandas-multiindex-dataframe-examples/?msclkid=fba45bb6c10e11eca1d376f5b274bab7 
    
    
    #df = df.droplevel(0, axis =1)
    return df

        


In [138]:
df = get_coin_info(five_ids_to_try,  config.coin_api_key)

In [139]:
df

,id,name,symbol,slug,num_market_pairs,date_added,max_supply,circulating_supply,total_supply,is_active,...,volume_change_24h,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap
0,1,Bitcoin,BTC,bitcoin,9357,2013-04-28T00:00:00.000Z,21000000,1.901909e+07,1.901909e+07,1,...,-9.2906,0.051088,-2.799836,-2.051057,-6.586973,5.289726,11.891771,7.533151e+11,40.7291,8.317755e+11
0,1027,Ethereum,ETH,ethereum,5659,2015-08-07T00:00:00.000Z,None,1.204864e+08,1.204864e+08,1,...,-8.2355,-0.099676,-1.394107,-2.203590,-1.058806,13.253093,21.616547,3.573333e+11,19.3177,3.573333e+11
0,825,Tether,USDT,tether,32227,2015-02-25T00:00:00.000Z,None,8.306920e+10,8.561395e+10,1,...,-11.5642,0.007491,0.002829,-0.001643,-0.016244,-0.027717,0.005706,8.308867e+10,4.4923,8.563401e+10
0,1839,BNB,BNB,bnb,813,2017-07-25T00:00:00.000Z,165116760,1.632770e+08,1.632770e+08,1,...,-2.8176,-0.227735,0.239985,-2.214106,0.567210,11.129498,12.376621,6.660225e+10,3.6009,6.735271e+10
0,3408,USD Coin,USDC,usd-coin,3790,2018-10-08T00:00:00.000Z,None,4.989235e+10,4.989235e+10,1,...,-4.1392,0.003145,0.015981,0.031693,-0.006726,0.048140,-0.021855,4.989603e+10,2.6977,4.989603e+10


,price,volume_24h,volume_change_24h,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,last_updated
4,420.658786,2.022702e+09,10.6199,0.63341,0.178291,-0.794199,6.117143,5.97843,-5.230828,6.868389e+10,3.5767,6.945782e+10,2022-04-21T00:47:00.000Z


In [103]:
#https://coinmarketcap.com/api/documentation/v1/#operation/getV2CryptocurrencyQuotesLatest
#now need to reference the coin ids and pull in data from the api accordinly 

config.coin_api_key

url = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/quotes/latest'
parameters = {
    'id': [1]
    }
headers = {
'Accepts': 'application/json',
'X-CMC_PRO_API_KEY': config.coin_api_key
}

session = Session()
session.headers.update(headers)
response = session.get(url, params=parameters)
data = json.loads(response.text)


In [104]:
#navigate json
data['data']
#read to DF to see what we have
cmc_df = pd.DataFrame.from_dict(data['data'], orient='index')
#need to explode this, then re join 
cmc_df['quote'][0]

{'USD': {'price': 41451.1353306875,
  'volume_24h': 27912601142.293224,
  'volume_change_24h': 11.804,
  'percent_change_1h': 0.09411368,
  'percent_change_24h': -0.11532505,
  'percent_change_7d': 0.78096388,
  'percent_change_30d': 1.10388286,
  'percent_change_60d': 3.42483063,
  'percent_change_90d': 1.74784875,
  'market_cap': 788289960909.4786,
  'market_cap_dominance': 41.0747,
  'fully_diluted_market_cap': 870473841944.44,
  'last_updated': '2022-04-21T00:20:00.000Z'}}

In [97]:
#expand nest columns
expanded_df = pd.DataFrame.from_dict(cmc_df['quote'][0], orient= 'index')
#showing USD as an index column, going to drop
expanded_df.reset_index(drop = True, inplace= True)
#can double check shape
expanded_df.shape
trial = cmc_df.drop(['quote','tags'], axis= 1)
#had to reset index or set it manually to match the expanded DF 
trial.set_index(pd.Index([0]), inplace= True)
#trial.reset_index(drop = True, inplace= True)

coin_price_df = pd.concat([trial,expanded_df], axis=1 )
coin_price_df.set_index(pd.Index([22]), inplace= True)

#going to try to append this to another dataframe
df = pd.DataFrame()

df = df.append(coin_price_df)
df.head()

,id,name,symbol,slug,num_market_pairs,date_added,max_supply,circulating_supply,total_supply,is_active,...,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,last_updated
22,1,Bitcoin,BTC,bitcoin,9340,2013-04-28T00:00:00.000Z,21000000,19017287,19017287,1,...,0.208146,0.288073,0.536021,0.829391,3.857871,0.565335,7.901044e+11,41.1164,8.724794e+11,2022-04-20T22:44:00.000Z


In [52]:
df

""


In [48]:
coin_price_df

,id,name,symbol,slug,num_market_pairs,date_added,max_supply,circulating_supply,total_supply,is_active,...,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,last_updated
0,1,Bitcoin,BTC,bitcoin,9340,2013-04-28T00:00:00.000Z,21000000,19017287,19017287,1,...,0.208146,0.288073,0.536021,0.829391,3.857871,0.565335,7.901044e+11,41.1164,8.724794e+11,2022-04-20T22:44:00.000Z


In [25]:
trial = cmc_df.drop(['quote','tags'], axis= 1)
trial.shape

(1, 16)

In [27]:
coin_price_df

,id,name,symbol,slug,num_market_pairs,date_added,max_supply,circulating_supply,total_supply,is_active,...,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,last_updated
1,1.0,Bitcoin,BTC,bitcoin,9340.0,2013-04-28T00:00:00.000Z,21000000.0,19017287.0,19017287.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.208146,0.288073,0.536021,0.829391,3.857871,0.565335,7.901044e+11,41.1164,8.724794e+11,2022-04-20T22:44:00.000Z


In [26]:
768620549906.7964/ 40426.70598566949

19012693.0

In [14]:
trial.index

Index(['USD'], dtype='object')